In [3]:
import csv
import math

from PIL import Image
import numpy as np
import tensorflow as tf
from keras import Model
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras.layers import Conv2D, Reshape, Dense, GlobalAveragePooling2D
from keras.utils import Sequence
from keras.optimizers import Adam
from keras.backend import epsilon


In [66]:
# 0.35, 0.5, 0.75, 1.0
ALPHA = 1.0

# 96, 128, 160, 192, 224
IMAGE_SIZE = 96

EPOCHS = 200
BATCH_SIZE = 1
PATIENCE = 50

MULTI_PROCESSING = True
THREADS = 4
TRAIN_CSV = "./tbtrain.csv"
CLASSES=2

In [73]:
class DataGenerator(Sequence):

    def __init__(self, csv_file):
        self.paths = []
        with open(csv_file, "r") as file:
            self.coords = np.zeros((sum(1 for line in file), 4 + CLASSES))
            file.seek(0)
            reader = csv.reader(file, delimiter=",")
            for index, row in enumerate(reader):
                for i, r in enumerate(row[1:7]):
                    row[i+1] = int(r)
                    
                path, image_height, image_width, x0, y0, x1, y1, _, class_id = row
                self.coords[index, min(4 + int(class_id), self.coords.shape[1]-1)] = 1
                try:
                    
                    self.coords[index, 0] = x0 * IMAGE_SIZE / image_width
                    self.coords[index, 1] = y0 * IMAGE_SIZE / image_height
                    self.coords[index, 2] = (x1 - x0) * IMAGE_SIZE / image_width
                    self.coords[index, 3] = (y1 - y0) * IMAGE_SIZE / image_height 
                   
                except:
                    self.coords[index, 0] = 0
                    self.coords[index, 1] = 0
                    self.coords[index, 2] = 0
                    self.coords[index, 3] = 0
                self.paths.append(path)
        print(len(self.coords),self.paths)

    def __len__(self):
        return math.ceil(len(self.coords) / BATCH_SIZE)

    def __getitem__(self, idx):
        batch_paths = self.paths[idx * BATCH_SIZE:(idx + 1) * BATCH_SIZE]
        batch_coords = self.coords[idx * BATCH_SIZE:(idx + 1) * BATCH_SIZE]

        batch_images = np.zeros((len(batch_paths), IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.float32)
        for i, f in enumerate(batch_paths):
            img = Image.open(f)
            img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
            img = img.convert('RGB')

            batch_images[i] = preprocess_input(np.array(img, dtype=np.float32))
            img.close()

        return batch_images, [batch_coords[...,:4], batch_coords[...,4:]]

In [82]:
def create_model(trainable=False):
    model = MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, alpha=ALPHA)

    # to freeze layers
    for layer in model.layers:
        layer.trainable = trainable
    model.layers[-2].trainable=True
    model.layers[-1].trainable=True
    out = model.layers[-1].output

    x = Conv2D(4, kernel_size=3)(out)
    x = Reshape((4,), name="coords")(x)

    y = GlobalAveragePooling2D()(out)
    y = Dense(CLASSES, name="classes", activation="softmax")(y)

    return Model(inputs=model.input, outputs=[x, y])

In [83]:
def log_mse(y_true, y_pred):
    return tf.reduce_mean(tf.log1p(tf.squared_difference(y_pred, y_true)), axis=-1)

def focal_loss(alpha=0.9, gamma=2):
      def focal_loss_with_logits(logits, targets, alpha, gamma, y_pred):
        weight_a = alpha * (1 - y_pred) ** gamma * targets
        weight_b = (1 - alpha) * y_pred ** gamma * (1 - targets)
    
    
        return (tf.log1p(tf.exp(-tf.abs(logits))) + tf.nn.relu(-logits)) * (weight_a + weight_b) + logits * weight_b 

      def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        logits = tf.log(y_pred / (1 - y_pred))

        loss = focal_loss_with_logits(logits=logits, targets=y_true, alpha=alpha, gamma=gamma, y_pred=y_pred)

        return tf.reduce_mean(loss)

      return loss

In [84]:
model = create_model()
# model.summary()



In [29]:
# import pandas as pd
# df = pd.read_csv('tbtrain.csv')
# df.head()

In [85]:
train_datagen = DataGenerator(TRAIN_CSV)

98 ['./tb images/Anomaly/anomaly1.jpeg', './tb images/Anomaly/anomaly2.jpeg', './tb images/Anomaly/anomaly3.jpeg', './tb images/Anomaly/anomaly4.jpeg', './tb images/Anomaly/anomaly5.jpeg', './tb images/Anomaly/anomaly6.jpeg', './tb images/Anomaly/anomaly7.jpeg', './tb images/Anomaly/anomaly9.jpeg', './tb images/Anomaly/anomaly10.jpeg', './tb images/Anomaly/anomaly11.jpeg', './tb images/Anomaly/anomaly12.jpeg', './tb images/Anomaly/anomaly13.jpeg', './tb images/Anomaly/anomaly14.jpeg', './tb images/Anomaly/anomaly15.jpeg', './tb images/Anomaly/anomaly16.jpeg', './tb images/Anomaly/anomaly17.jpeg', './tb images/Anomaly/anomaly18.jpeg', './tb images/Anomaly/anomaly19.jpeg', './tb images/Anomaly/anomaly20.jpeg', './tb images/Anomaly/anomaly21.jpeg', './tb images/Anomaly/anomaly22.jpeg', './tb images/Anomaly/anomaly23.jpeg', './tb images/Anomaly/anomaly24.jpeg', './tb images/Anomaly/anomaly25.jpeg', './tb images/Anomaly/anomaly26.jpeg', './tb images/Anomaly/anomaly27.jpeg', './tb images/Ano

In [88]:

# validation_datagen = Validation(generator=DataGenerator(VALIDATION_CSV))



optimizer = Adam(lr=1e-2, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.1, amsgrad=False)
model.compile(loss={"coords" : log_mse, "classes" : focal_loss()}, loss_weights={"coords" : 2, "classes" : 1}, optimizer=optimizer, metrics=[])

In [89]:
import pandas as pd
df=pd.read_csv(TRAIN_CSV)
df.head()

,./tb images/Anomaly/anomaly1.jpeg,1798,2637,317,659,711,964,anomaly,1
0,./tb images/Anomaly/anomaly2.jpeg,1831,2569,1928,870,2357,1141,anomaly,1
1,./tb images/Anomaly/anomaly3.jpeg,1725,2663,2006,651,2287,886,anomaly,1
2,./tb images/Anomaly/anomaly4.jpeg,472,743,276,240,316,282,anomaly,1
3,./tb images/Anomaly/anomaly5.jpeg,472,743,274,190,317,229,anomaly,1
4,./tb images/Anomaly/anomaly6.jpeg,472,743,422,189,473,230,anomaly,1


In [90]:
 model.fit_generator(generator=train_datagen,
                        epochs=150)

Epoch 1/150
98/98 [==============================] - 15s 150ms/step - loss: 10.1836 - coords_loss: 4.8016 - classes_loss: 0.5804
Epoch 2/150
98/98 [==============================] - 12s 124ms/step - loss: 9.7821 - coords_loss: 4.8436 - classes_loss: 0.0949
Epoch 3/150
98/98 [==============================] - 12s 125ms/step - loss: 9.3299 - coords_loss: 4.6319 - classes_loss: 0.0660
Epoch 4/150
98/98 [==============================] - 12s 127ms/step - loss: 9.0841 - coords_loss: 4.5123 - classes_loss: 0.0595
Epoch 5/150
98/98 [==============================] - 13s 130ms/step - loss: 8.6233 - coords_loss: 4.2848 - classes_loss: 0.0537
Epoch 6/150
98/98 [==============================] - 12s 127ms/step - loss: 8.0807 - coords_loss: 4.0129 - classes_loss: 0.0550
Epoch 7/150
98/98 [==============================] - 12s 127ms/step - loss: 7.5831 - coords_loss: 3.7639 - classes_loss: 0.0553
Epoch 8/150
98/98 [==============================] - 13s 128ms/step - loss: 7.1649 - coords_loss: 3.557

98/98 [==============================] - 12s 121ms/step - loss: 2.8345 - coords_loss: 1.4064 - classes_loss: 0.0217
Epoch 128/150
98/98 [==============================] - 12s 121ms/step - loss: 2.8270 - coords_loss: 1.4027 - classes_loss: 0.0217
Epoch 129/150
98/98 [==============================] - 12s 120ms/step - loss: 2.8199 - coords_loss: 1.3992 - classes_loss: 0.0216
Epoch 130/150
98/98 [==============================] - 12s 123ms/step - loss: 2.8131 - coords_loss: 1.3958 - classes_loss: 0.0215
Epoch 131/150
98/98 [==============================] - 12s 121ms/step - loss: 2.8068 - coords_loss: 1.3926 - classes_loss: 0.0215
Epoch 132/150
98/98 [==============================] - 12s 121ms/step - loss: 2.8012 - coords_loss: 1.3899 - classes_loss: 0.0214
Epoch 133/150
98/98 [==============================] - 12s 121ms/step - loss: 2.7960 - coords_loss: 1.3872 - classes_loss: 0.0215
Epoch 134/150
98/98 [==============================] - 12s 120ms/step - loss: 2.7902 - coords_loss: 1.38

In [65]:
model.save_weights("model-0.51.h5")

NameError: name 'model' is not defined

# TB classification using SVM

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import os


In [31]:
dataset = "./tb images/"
categories = ["intestinal", "peritoneal","mesenteric"]

In [32]:
training_data = []

def create_training_data():
    for category in categories:  

        path = os.path.join(dataset,category) 
        class_num = categories.index(category)  
        
        for img in os.listdir(path): 
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE) 
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  
                #print(np.array(new_array).shape)
                training_data.append([new_array, class_num]) 
            except Exception as e:  
                pass
create_training_data()

In [32]:
# training_data

In [33]:
import random
random.shuffle(training_data)
IMG_SIZE=28

In [34]:
X = []
y = []
for data in training_data:
    X.append(data[0])
    y.append(data[1])
# print(y)
# print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))
X = np.array(X)
X=X.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y=np.array(y)

In [35]:

X = X/255.0
shape_of_x=X.shape
X=X.reshape(shape_of_x[0],shape_of_x[1],shape_of_x[2])
Y=Y.reshape(-1,1)


In [36]:
X=X.reshape(shape_of_x[0],shape_of_x[1]*shape_of_x[2])

In [37]:
from sklearn import datasets 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC 
svm = SVC(kernel='rbf', random_state=0, gamma=.01, C=1)
svm.fit(X,y)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [38]:
#testimage="tb images/intestinal/anomaly1.jpeg"
testimagae="./tb images/Anomaly/anomaly33.jpeg"

In [39]:
img_array = cv2.imread(testimage ,cv2.IMREAD_GRAYSCALE) 
new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  
new_array = new_array/255.0
test=new_array.reshape(1,new_array.shape[0]*new_array.shape[1])


In [40]:
if svm.predict(test)==0:
    print("the type of tb is -> intestinal")
elif svm.predict(test)==1:
    print(" the type of tb is -> peritenal")
else:
    print("the type of tb is -> messenteric")

the type of tb is -> intestinal


# Accuracy


In [26]:
from sklearn.metrics import accuracy_score

In [27]:
y_pred=svm.predict(X)


In [28]:
y_true=y

In [29]:
str(accuracy_score(y_true, y_pred)*100)+"%"

'100.0%'

# Confusion matrix

In [41]:
from sklearn.metrics import confusion_matrix

In [43]:
confusion_matrix(y_true,y_pred)

array([[3, 0],
       [0, 3]], dtype=int64)